In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [9]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
        )

    def forward(self, x):
        print("Original:", x.shape)
        x = self.conv1(x)
        print("First conv:", x.shape)
        x = self.conv2(x)
        print("Second conv:", x.shape)
        return x

x = torch.randn(64, 3, 224, 224)
model = ResNet()
y = model(x)
y.shape

Original: torch.Size([64, 3, 224, 224])
First conv: torch.Size([64, 64, 54, 54])
Second conv: torch.Size([64, 64, 50, 50])


torch.Size([64, 64, 50, 50])

In [8]:
x = torch.randn(64, 3, 224, 224)
model = ResNet()
y = model(x)
y.shape

torch.Size([64, 3, 224, 224])
torch.Size([64, 64, 54, 54])
torch.Size([64, 64, 50, 50])


torch.Size([64, 64, 50, 50])